# Сравнение пользователей музыкального сервиса среди двух крупных городов.

**Цель исследования** — проверю *три гипотезы*:
1. Активность пользователей зависит от дня недели. 
2. В первый будний день утром в столице России преобладают определенные виды жанов музыки, а в СПб — другие. Так же и с вечером пятницы - в зависимости от города преобладают разные жанры . 
3. Пользователи из двух разных крупных городов предпочитают разные музыкальные жанры. В столице -  чаще слушают поп-музыку, в СПб — русский рэп.

**Ход исследования**

Проведем первичный анализ данных.
Проверим данные на аномалии и оценим их влияние на исследование. 
На этапе обрабтки - исправление критичных аномалий данных.
 
**Таким образом, исследование пройдёт в три этапа:**

 1. Знакомство с данными.
 2. Предобработка данных.
 3. Проверка гипотез.

## Знакомство с данными.

In [58]:
# импорт библиотеки pandas

import pandas as pd 

In [59]:
# чтение файла с данными и сохранение в df

df = pd.read_csv('yandex_music_project.csv')  

In [60]:
df.head(10)  

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
8,8FA1D3BE,И вновь продолжается бой,NaN,ruspop,Moscow,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Saint-Petersburg,21:20:49,Wednesday


In [61]:
df.info() # получение общей информации о данных в таблице df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


Итак, в таблице семь столбцов. Тип данных во всех столбцах — `object`.

Согласно документации к данным:
* `userID` — идентификатор пользователя;
* `Track` — название трека;  
* `artist` — имя исполнителя;
* `genre` — название жанра;
* `City` — город пользователя;
* `time` — время начала прослушивания;
* `Day` — день недели.

В названиях колонок видны три нарушения стиля:
1. Строчные буквы сочетаются с прописными.
2. Встречаются пробелы.
3. Столбцы лучше переписать "змеиным" регистре.
4. Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.


**Выводы**

В каждой строке таблицы — данные о прослушанном треке. Часть колонок описывает саму композицию: название, исполнителя и жанр. Остальные данные рассказывают о пользователе: из какого он города, когда он слушал музыку. 

Предварительно можно утверждать, что, данных достаточно для проверки гипотез. Но встречаются пропуски в данных, а в названиях колонок — расхождения с хорошим стилем.

Чтобы двигаться дальше, нужно устранить проблемы в данных.

## Предобработка данных

Исправлю стиль в заголовках столбцов, исключю пропуски. Затем проверю данные на дубликаты.

### Стиль заголовков

Выведу на экран названия столбцов:

In [62]:
df.columns  # перечень названий столбцов таблицы df

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

Приведу названия в соответствие с хорошим стилем:
* несколько слов в названии запишу в «змеином_регистре»,
* все символы сделаю строчными,
* устраню пробелы.

Для этого переименую колонки так:
* `'  userID'` → `'user_id'`;
* `'Track'` → `'track'`;
* `'  City  '` → `'city'`;
* `'Day'` → `'day'`.

In [63]:
# переименование столбцов:

df = df.rename(columns={'  userID': 'user_id', 'Track': 'track', 'artist': 'artist', 'genre': 'genre', 
                        '  City  ': 'city', 'time': 'time', 'Day': 'day'})

Проверю результат. Для этого ещё раз выведу на экран названия столбцов:

In [64]:
print(df.columns) # проверка результатов - перечень названий столбцов

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')


### Пропуски значений

Сначала посчитаю, сколько в таблице пропущенных значений:

In [65]:
print(df.isna().sum()) # подсчёт пропусков

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64


Не все пропущенные значения влияют на исследование. Так в `track` и `artist` пропуски не важны для вашей работы. Достаточно заменить их явными обозначениями.

Но пропуски в `genre` могут помешать сравнению музыкальных вкусов в Moscow и SPb. На практике было бы правильно установить причину пропусков и восстановить данные. Такой возможности нет в учебном проекте. Придётся:
* заполнить и эти пропуски явными обозначениями,
* оценить, насколько они повредят расчётам. 

Заменю пропущенные значения в столбцах `track`, `artist` и `genre` на строку `'unknown'`. Для этого создайте список `columns_to_replace`, переберите его элементы циклом `for` и для каждого столбца выполните замену пропущенных значений:

In [66]:
columns_to_replace =  ['track', 'artist', 'genre']
for col in columns_to_replace:
    df[col] = df[col].fillna('unknown')
    # перебор названий столбцов в цикле и замена пропущенных значений на 'unknown'

Убеждаюсь, что в таблице не осталось пропусков. Для этого ещё раз посчитаю пропущенные значения.

In [67]:
print(df.isna().sum()) # подсчёт пропусков


user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64


### Дубликаты

Посчитайте явные дубликаты в таблице одной командой:

In [68]:
df.duplicated().sum()  # подсчёт явных дубликатов

3826

Вызову специальный метод `pandas`, чтобы удалить явные дубликаты:

In [69]:
df = df.drop_duplicates().reset_index(drop=True) # удаление явных дубликатов (с удалением старых индексов и формированием новых)

Ещё раз посчитаю явные дубликаты в таблице — убеждаюсь, что полностью от них избавилась:

In [70]:
df.duplicated().sum() # проверка на отсутствие дубликатов

0

Теперь избавлюсь от неявных дубликатов в колонке `genre`. Например, название одного и того же жанра может быть записано немного по-разному. Такие ошибки тоже повлияют на результат исследования.

Выведу на экран список уникальных названий жанров, отсортированный в алфавитном порядке. Для этого:
* извлеку нужный столбец датафрейма, 
* применю к нему метод сортировки,
* для отсортированного столбца вызову метод, который вернёт уникальные значения из столбца.

In [71]:
df['genre'].sort_values().unique() # Просмотр уникальных названий жанров

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

Просмотрю список и найду неявные дубликаты:

In [72]:
# Функция для замены неявных дубликатов:

def replace_wrong_genres(wrong_genres, correct_genre): 
    for genre in wrong_genres:
        df['genre'] = df['genre'].replace(genre, correct_genre)
    return df

Вызову `replace_wrong_genres()` и передам ей такие аргументы, чтобы она устранила неявные дубликаты: вместо `hip`, `hop` и `hip-hop` в таблице должно быть значение `hiphop`:

In [73]:
df = replace_wrong_genres(['hip', 'hop', 'hip-hop'], 'hiphop')  # Устранение неявных дубликатов

Проверяю, что заменили неправильные названия:

*   hip
*   hop
*   hip-hop

Выведу отсортированный список уникальных значений столбца `genre`:

In [74]:
df['genre'].unique() # Проверка на неявные дубликаты

array(['rock', 'pop', 'folk', 'dance', 'rusrap', 'ruspop', 'world',
       'electronic', 'unknown', 'alternative', 'children', 'rnb',
       'hiphop', 'jazz', 'postrock', 'latin', 'classical', 'metal',
       'reggae', 'tatar', 'blues', 'instrumental', 'rusrock', 'dnb',
       'türk', 'post', 'country', 'psychedelic', 'conjazz', 'indie',
       'posthardcore', 'local', 'avantgarde', 'punk', 'videogame',
       'techno', 'house', 'christmas', 'melodic', 'caucasian',
       'reggaeton', 'soundtrack', 'singer', 'ska', 'shanson', 'ambient',
       'film', 'western', 'rap', 'beats', "hard'n'heavy", 'progmetal',
       'minimal', 'contemporary', 'new', 'soul', 'holiday', 'german',
       'tropical', 'fairytail', 'spiritual', 'urban', 'gospel', 'nujazz',
       'folkmetal', 'trance', 'miscellaneous', 'anime', 'hardcore',
       'progressive', 'chanson', 'numetal', 'vocal', 'estrada', 'russian',
       'classicmetal', 'dubstep', 'club', 'deep', 'southern', 'black',
       'folkrock', 'fitness'

**Выводы**

В ходе предобработки обнаружила три проблемы в данных:

- нарушения в стиле заголовков,
- пропущенные значения,
- дубликаты — явные и неявные.

Исправила заголовки, чтобы упростить работу с таблицей. Без дубликатов исследование станет более точным.

Пропущенные значения заменила на `'unknown'`. Ещё предстоит увидеть, не повредят ли исследованию пропуски в колонке `genre`.

Теперь можно перейти к проверке гипотез. 

## Проверка гипотез

### Сравнение поведения пользователей двух столиц

Первая гипотеза утверждает, что пользователи по-разному слушают музыку в двух крупных городах. Проверю это предположение по данным о трёх днях недели — понедельнике, среде и пятнице. Для этого:

* Разделю пользователей двух разных крупных городов.
* Сравню, сколько треков послушала каждая группа пользователей в понедельник, среду и пятницу.

Для тренировки сначала выполню каждый из расчётов по отдельности. 

Оценю активность пользователей в каждом городе. Сгруппирую данные по городу и посчитаю прослушивания в каждой группе.

In [75]:
listening_music_moscow = df.loc[df['city'] == 'Moscow'] # Подсчёт прослушиваний в каждом городе
listening_music_moscow

,user_id,track,artist,genre,city,time,day
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
6,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
8,8FA1D3BE,И вновь продолжается бой,unknown,ruspop,Moscow,09:17:40,Friday
...,...,...,...,...,...,...,...
61247,83A474E7,I Worship Only What You Bleed,The Black Dahlia Murder,extrememetal,Moscow,21:07:12,Monday
61248,729CBB09,My Name,McLean,rnb,Moscow,13:32:28,Wednesday
61250,C5E3A0D5,Jalopiina,unknown,industrial,Moscow,20:09:26,Friday
61251,321D0506,Freight Train,Chas McDevitt,rock,Moscow,21:43:59,Friday


In [76]:
listening_music_spb = df.loc[df['city'] == 'Saint-Petersburg'] # Подсчёт прослушиваний в каждом городе
listening_music_spb

,user_id,track,artist,genre,city,time,day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
9,E772D5C0,Pessimist,unknown,dance,Saint-Petersburg,21:20:49,Wednesday
...,...,...,...,...,...,...,...
61239,D94F810B,Theme from the Walking Dead,Proyecto Halloween,film,Saint-Petersburg,21:14:40,Monday
61240,BC8EC5CF,Red Lips: Gta (Rover Rework),Rover,electronic,Saint-Petersburg,21:06:50,Monday
61241,29E04611,Bre Petrunko,Perunika Trio,world,Saint-Petersburg,13:56:00,Monday
61242,1B91C621,(Hello) Cloud Mountain,sleepmakeswaves,postrock,Saint-Petersburg,09:22:13,Monday


В первом городе прослушиваний больше, чем в во втором. Из этого не следует, что пользователи первого города чаще слушают музыку. Просто самих пользователей в первом городе больше.

Теперь сгруппирую данные по дню недели и подсчитаю прослушивания в понедельник, среду и пятницу. Учту, что в данных есть информация только о прослушиваниях только за эти дни.

In [77]:
days_moscow = df.loc[df['city'] == 'Moscow']
days_sbp = df.loc[df['city'] == 'Saint-Petersburg'] # Подсчёт прослушиваний в каждый из трёх дней

In [78]:
week_days_moscow = days_moscow.groupby('day', as_index=False)['user_id'].count()
week_days_moscow

,day,user_id
0,Friday,15945
1,Monday,15740
2,Wednesday,11056


In [79]:
week_days_spb = days_sbp.groupby('day', as_index=False)['user_id'].count()
week_days_spb

,day,user_id
0,Friday,5895
1,Monday,5614
2,Wednesday,7003


В среднем пользователи из двух городов менее активны по средам. Но картина может измениться, если рассмотреть каждый город в отдельности.

Мы видели, как работает группировка по городу и по дням недели. Теперь напишу функцию, которая объединит два эти расчёта.

Создам функцию `number_tracks()`, которая посчитает прослушивания для заданного дня и города. Ей понадобятся два параметра:
* день недели,
* название города.

В функции сохраню в переменную строки исходной таблицы, у которых значение:
  * в колонке `day` равно параметру `day`,
  * в колонке `city` равно параметру `city`.

Для этого применю последовательную фильтрацию с логической индексацией.

Затем посчитаю значения в столбце `user_id` получившейся таблицы. Результат сохраню в новую переменную. Верну эту переменную из функции.

In [80]:
 # создание функции number_tracks():
def number_track(day, city):    
    """
 Функция для подсчёта прослушиваний для конкретного города и дня.
 С помощью последовательной фильтрации с логической индексацией она 
 сначала получит из исходной таблицы строки с нужным днём,
 затем из результата отфильтрует строки с нужным городом,
 методом count() посчитает количество значений в колонке user_id. 
 Это количество функция вернёт в качестве результата.    
    """
    track_list = df.loc[df['day'] == day] 
    track_list = track_list.loc[track_list['city'] == city]
    track_list_count = track_list['user_id'].count()
    return track_list_count

Вызову функцию `number_tracks()` шесть раз, меняя значение параметров — так, чтобы получить данные для каждого города в каждый из трёх дней.

In [81]:
number_track('Monday', 'Moscow')  # количество прослушиваний в Москве по понедельникам

15740

In [82]:
number_track('Monday', 'Saint-Petersburg') # количество прослушиваний в Санкт-Петербурге по понедельникам

5614

In [83]:
number_track('Wednesday', 'Moscow')  # количество прослушиваний в Москве по средам

11056

In [84]:
number_track('Wednesday', 'Saint-Petersburg') # количество прослушиваний в Санкт-Петербурге по средам

7003

In [85]:
number_track('Friday', 'Moscow') # количество прослушиваний в Москве по пятницам

15945

In [86]:
number_track('Friday', 'Saint-Petersburg')  # количество прослушиваний в Санкт-Петербурге по пятницам

5895

Создам c помощью конструктора `pd.DataFrame` таблицу, где:
* названия колонок — `['city', 'monday', 'wednesday', 'friday']`;
* данные — результаты, которые вы получили с помощью `number_tracks`.

In [87]:
table_result = pd.DataFrame(data={'city': ['Moscow', 'Saint-Petersburg'], 'monday': [15740, 5614], 'wednesday': [11056, 7003], 'friday': [15945, 5895]})   # Таблица с результатами

In [88]:
table_result

,city,monday,wednesday,friday
0,Moscow,15740,11056,15945
1,Saint-Petersburg,5614,7003,5895


**Выводы**

Данные показывают разницу поведения пользователей:

- В Москве *пик* прослушиваний приходится на *понедельник и пятницу*, а в *среду* заметен *спад*.
- В Питере, наоборот, больше слушают музыку по *средам*. Активность в понедельник и пятницу здесь почти в равной мере уступает среде.

Значит, данные говорят в пользу первой гипотезы.

### Музыка в начале и в конце недели

Согласно второй гипотезе, утром в понедельник в столице преобладают одни жанры, а в Питере — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.

Сохраню таблицы с данными в две переменные:
* по Москве — в `moscow_general`;
* по Санкт-Петербургу — в `spb_general`.

In [89]:
moscow_general = df.loc[df['city'] == 'Moscow']  # получение таблицы moscow_general из тех строк таблицы df, 
# для которых значение в столбце 'city' равно 'Moscow'

In [90]:
spb_general = df.loc[df['city'] == 'Saint-Petersburg']    # получение таблицы spb_general из тех строк таблицы df,
# для которых значение в столбце 'city' равно 'Saint-Petersburg'

Создам функцию `genre_weekday()` с четырьмя параметрами:
* таблица (датафрейм) с данными,
* день недели,
* начальная временная метка в формате 'hh:mm', 
* последняя временная метка в формате 'hh:mm'.

Функция вернет информацию о топ-10 жанров тех треков, которые прослушивали в указанный день, в промежутке между двумя отметками времени.

In [91]:
def genre_weekday(table, day, time1, time2): 
    '''
 Функция возвращает информацию о самых популярных жанрах в указанный день в
 заданное время:
 1) в переменную genre_df сохраняются те строки переданного датафрейма table, для
    которых одновременно:
    - значение в столбце day равно значению аргумента day
    - значение в столбце time больше значения аргумента time1
    - значение в столбце time меньше значения аргумента time2
    Используйте последовательную фильтрацию с помощью логической индексации.
 2) сгруппировать датафрейм genre_df по столбцу genre, взять один из его
    столбцов и посчитать методом count() количество записей для каждого из
    присутствующих жанров, получившийся Series записать в переменную
    genre_df_count
 3) отсортировать genre_df_count по убыванию встречаемости и сохранить
    в переменную genre_df_sorted
4) вернуть Series из 10 первых значений genre_df_sorted, это будут топ-10
    популярных жанров (в указанный день, в заданное время)
    '''
    genre_df = table.loc[table['day'] == day]
    genre_df = genre_df.loc[genre_df['time'] > time1]
    genre_df = genre_df.loc[genre_df['time'] < time2]
    genre_df_count = genre_df.groupby('genre')['genre'].count()
    genre_df_sorted =  genre_df_count.sort_values(ascending=False)
    return genre_df_sorted.head(10)


Cравню результаты функции `genre_weekday()` для Москвы и СПб в понедельник утром (с 7:00 до 11:00) и в пятницу вечером (с 17:00 до 23:00):

In [92]:
genre_weekday(moscow_general, 'Monday', '07:00', '11:00')  
# вызов функции для утра понедельника в Москве (вместо df — таблица moscow_general)
# объекты, хранящие время, являются строками и сравниваются как строки
# пример вызова: genre_weekday(moscow_general, 'Monday', '07:00', '11:00')

genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
Name: genre, dtype: int64

In [93]:
genre_weekday(spb_general, 'Monday', '07:00', '11:00')  # вызов функции для утра понедельника в Петербурге (вместо df — таблица spb_general)

genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: genre, dtype: int64

In [94]:
genre_weekday(moscow_general, 'Friday', '17:00', '23:00') # вызов функции для вечера пятницы в Москве

genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: genre, dtype: int64

In [95]:
genre_weekday(spb_general, 'Friday', '17:00', '23:00') # вызов функции для вечера пятницы в Петербурге

genre
pop            256
electronic     216
rock           216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: genre, dtype: int64

**Выводы**

Если сравнить топ-10 жанров в понедельник утром, можно сделать такие выводы:

1. В Москве и Петербурге слушают похожую музыку. Единственное отличие — в московский рейтинг вошёл жанр *“world”*, а в петербургский — *джаз* и *классика*.

2. В Москве пропущенных значений оказалось так много, что значение `'unknown'` заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных и угрожают достоверности исследования.

Вечер пятницы не меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым.

Таким образом, *вторая гипотеза* подтвердилась лишь *частично*:
* Пользователи слушают похожую музыку в начале недели и в конце.
* Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.

Однако пропуски в данных ставят под сомнение этот результат. В Москве их так много, что рейтинг топ-10 мог бы выглядеть иначе, если бы не утерянные  данные о жанрах.

### Жанровые предпочтения в Москве и Петербурге

Гипотеза: СПб — столица рэпа, музыку этого жанра там слушают чаще, чем в столице России. А Moscow — большой город контрастов, в котором, преобладает поп-музыка.

Сгруппирую таблицу `moscow_general` по жанру и посчитаю прослушивания треков каждого жанра методом `count()`. Затем отсортирую результат в порядке убывания и сохраню его в таблице `moscow_genres`.

In [96]:
moscow_genres = moscow_general.groupby('genre')['track'].count()  # одной строкой: группировка таблицы moscow_general по столбцу 'genre', # подсчёт числа значений 'genre' в этой группировке методом count(),
moscow_genres = moscow_genres.sort_values(ascending=False)  # сортировка получившегося Series в порядке убывания и сохранение в moscow_genres

Выведу на экран первые десять строк `moscow_genres`:

In [97]:
moscow_genres.head(10)   # просмотр первых 10 строк moscow_genres

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: track, dtype: int64

Теперь повторю то же и для Петербурга.

Сгруппирую таблицу `spb_general` по жанру. Посчитаю прослушивания треков каждого жанра. Результат отсортирую в порядке убывания и сохраню в таблице `spb_genres`:

In [98]:
# одной строкой: группировка таблицы spb_general по столбцу 'genre',
# подсчёт числа значений 'genre' в этой группировке методом count():
spb_genres = spb_general.groupby('genre')['track'].count() 


# сортировка получившегося Series в порядке убывания и сохранение в spb_genres:
spb_genres = spb_genres.sort_values(ascending=False)

Выведите на экран первые десять строк `spb_genres`:

In [99]:
spb_genres.head(10)  # просмотр первых 10 строк spb_genres

genre
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: track, dtype: int64

**Выводы**

Гипотеза *частично* подтвердилась:
* Поп-музыка — самый популярный жанр в столице, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
* Вопреки ожиданиям, рэп одинаково популярен в Москве и Питере. 


## Итоги исследования

Я проверила три гипотезы и установила:

1. День недели по-разному влияет на активность пользователей в Москве и Петербурге. 

*Первая гипотеза полностью подтвердилась.*

2. Музыкальные предпочтения *не сильно меняются в течение недели* — будь то Москва или СПб. Небольшие различия заметны в начале недели, по понедельникам:
* в столице слушают музыку жанра “world”,
* в Питере — джаз и классику.

Таким образом, *вторая гипотеза подтвердилась лишь отчасти*. Этот результат мог оказаться иным, если бы не пропуски в данных.

3. Во вкусах пользователей Москвы и Петербурга больше общего чем различий. Вопреки ожиданиям, предпочтения жанров в Петербурге напоминают московские.

*Третья гипотеза не подтвердилась*. Если различия в предпочтениях и существуют, на основной массе пользователей они незаметны.

